# Import libraries, set options, connect to DB

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
import mapToPoly
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
pd.set_option('display.max_colwidth', -1)
import csv

# Connect to database

In [2]:
# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import MixpanelMap, Base

dbURL = os.environ['dbURL']

engine = create_engine(dbURL)
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


True
True


# Read in data as pandas data frame, selecting only certain fields

In [3]:
# df = pd.read_csv('modified_iten.csv', usecols = fields, dtype={"selectedDestination_id" : "str"})
df = pd.read_csv('modified_iten.csv', dtype={"selectedDestination_id" : "str"})

# Get column names as a list, adding commas and quotation marks
colNames = list(df.columns)
colNamesString = '","'.join(colNames)

colNamesString = '"' + colNamesString + '"'

# Combine all of the columns into one
# separating by quotation marks and comma
df['concat'] = pd.Series(df.fillna('').values.tolist()).map(lambda x: '","'.join(map(str,x)))

# Add commas at the beggining and the end
df['concat'] = '"' + df['concat'].astype(str)
df['concat'] = df['concat'].astype(str) + '"'

# Add column names at the beggining
df['concat'] = colNamesString + "\n" + df['concat'].astype(str)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,4,14,17,22,37,38,39,42,43,45,51,53,55,57,61,63,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df[['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time', 'user_id', 'start_from_latlng', 'concat']]

# Wrange field: destinationIDs

In [5]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

## Remove the 2 cases where the string says null
## Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

## Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)

# Wrangle field: numItenerariesReturned

In [6]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [7]:
#Convert this field to an integer replacing all NA's with zero
# This gets rid of the trailing zeros
df.selectedDestination_name.fillna("", inplace = True)

# Wrangle Field: departureDate

In [8]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))

# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']

print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
37204
number of rows after removal of anamoulous departureDate cases
37196


# Wrangle Field: distinctID

In [9]:
# Create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)
vc = df.primary_key.value_counts()
unique_keys = df.primary_key.unique()

# Wrangle user_id field

In [10]:
df.dtypes

distinct_id                 object 
numItinerariesReturned      int64  
departureDate               object 
startFromLocation           object 
selectedDestination_id      int64  
selectedDestination_name    object 
time                        int64  
user_id                     float64
start_from_latlng           object 
concat                      object 
primary_key                 object 
dtype: object

In [11]:
# This gets rid of the railing zeros and all of the nas are just
# blank

df['user_id'] = df['user_id'].fillna(0).astype(np.int64)

#Convert to string to be consistent with other fields in database
df['user_id'] = df['user_id'].astype(str)


In [12]:
df["start_from_latlng"] = df["start_from_latlng"].fillna("")
df["start_from_latlng"] = df["start_from_latlng"].astype(str)


# Create a subset of the datle with sample method to test geocode and database entry logic

In [13]:
# Create a random sample of the database, these entries will be added to the database in the next section
len(df)
sampleDf = df.tail(40000)

# Output this random sample
# sampleDf.head(len(sampleDf))

## Read in the destination data to allow the possibility to pull the correct names

 


In [14]:
# sampleDf.dtypes

# Loop through the rows in the dataframe, geocode, add entry to database

In [15]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in df.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(MixpanelMap.distinctkey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # In the case that the desintaiton ID is zero, this means it was an NA
        # So don't even try to geocode
        
        if row["selectedDestination_id"] != 0:
        
            # Check to see if there is already a latlng value
            # If there isn't, geocode
            if not row['start_from_latlng']:
            
                # Try to run the geocode function that returns a dictionary of information
                try:
                    geocodeInfo = geocode(locationToGeocode)
                    # If geocoding works, set valid to tre
                    valid = True
                    lat = geocodeInfo['lat']
                    lng = geocodeInfo['lng']
                    adressPretty=geocodeInfo['formatted_address']
                    
                # If the geocode function doesn't work set valid to false
                except:
                    valid = False
            
            # IF there already is a lat lng value, in the case of the newer rows
            else:
                
                # In this case we aren't geocoding so we won't get a pretty address
                adressPretty=''
                
                lat = row['start_from_latlng'].split(',')[0]
                lng = row['start_from_latlng'].split(',')[1]
                lat = float(lat)
                lng = float(lng)
                valid = True
        
        # In the case that the selected Destination ID is 0 then set valid to false
        else:
            valid=False

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:

            
            # Mapp the gps coordinates returned to the zip code polygons
            zipCodeInfo = mapToPoly(lat, lng, 'postal')
            
            
            if zipCodeInfo:
                zipCodeMapped = zipCodeInfo[0]
                region = zipCodeInfo[1]
            else:
                zipCodeMapped = 'outsideRegion'
                zipCodeMapped = 'outsideRegion'
            
            barrioInfo = mapToPoly(lat, lng, 'barrio')
            print(barrioInfo)
            
            if barrioInfo:
                barrioMapped = barrioInfo
            else:
                barrioMapped = 'outsideRegion'
                
            ## Get selected Destination Names
            # Pull the selected destination name
            selectedDestinationName = row["selectedDestination_name"]
            
            #### Just need to fix this for the cases when I can't get a geocodeinfo object #####
            
            databaseEntry = MixpanelMap(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=selectedDestinationName,
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=adressPretty,
                                      lat=lat,
                                      lng=lng,
                                      postalcodemapped=zipCodeMapped,
                                      barriomapped=barrioMapped,
                                      userid=row["user_id"],
                                      region=region,
                                      valid=valid,
                                      datablob=row["concat"])
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = MixpanelMap(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=row["selectedDestination_name"],
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      userid=row["user_id"],
                                      valid=valid,
                                      datablob=row['concat'])

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

...

None
None
CBD
271869
271850
{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
CBD
274674
271869
{'formatted_address': '1350 Spring St NW #1, Atlanta, GA 30309, USA', 'lat': 33.791638, 'lng': -84.389488, 'postalCode': '30309'}
None
CBD
{'formatted_address': '1350 Spring St NW #1, Atlanta, GA 30309, USA', 'lat': 33.791638, 'lng': -84.389488, 'postalCode': '30309'}
None
CBD
CBD
CBD
CBD
CBD
CBD
CBD
271849
271849
271849
None
271849
250149
271849
271869
272001
None
271849
250206
MP
MP
CBD
344028
344028
344013
344013
344013
344028
None
None
None
None
None
{'formatted_address': '3711 Fir St, Burnaby, BC V5G 2A4, Canada', 'lat': 49.2450331, 'lng': -123.023049, 'postalCode': 'V5G 2A4'}
None
271849
None
{'formatted_address': 'Timber Creek, North Creek, WA 98012, USA', 'lat': 47.8305, 'lng': -122.1712749, 'postalCode': '98012'}
None
250206
250206
{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
CBD
None
None
WE
271942
34402

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry alre